# W01B — Bronze/Raw: ingesta + trazabilidad + sanity checks (DDIA Cap. 1)

## Objetivo
- Garantizar que existe un **Raw** en `data/raw/` con nombre esperado
- Registrar **hash SHA-256** (trazabilidad)
- Consultar el CSV vía **VIEW raw_ps** (sin modificar el archivo)
- Ejecutar sanity checks y guardar evidencia en `artifacts/`

> Importante: En Semana 1 **no enseñamos SQL formal**. SQL aquí es instrumento de evidencia.


In [1]:
# Setup común (cross-platform) + detección de raíz del proyecto
import sys, time, json, hashlib, platform, subprocess
from pathlib import Path
import duckdb

def find_project_root(start: Path) -> Path:
    """Busca hacia arriba una carpeta que contenga `src/` y `data/`.
    Esto evita errores cuando el notebook se ejecuta desde notebooks/."""
    cur = start.resolve()
    for p in [cur] + list(cur.parents):
        if (p / "src").exists() and (p / "data").exists():
            return p
    # fallback: carpeta actual
    return cur

PROJECT_ROOT = find_project_root(Path.cwd())
print("PROJECT_ROOT:", PROJECT_ROOT)

DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
DOCS_DIR = PROJECT_ROOT / "docs"
for d in [RAW_DIR, ARTIFACTS_DIR, DOCS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

DB_PATH = DATA_DIR / "exoplanets.duckdb"
con = duckdb.connect(str(DB_PATH))

def show(path: Path):
    return {"exists": path.exists(), "path": str(path), "size_bytes": path.stat().st_size if path.exists() else None}

def sha256_file(path: Path, chunk_size: int = 1<<20) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def run_module(mod: str, *args: str):
    """Ejecuta `python -m ...` desde la raíz del proyecto para que `src.*` sea importable."""
    cmd = [sys.executable, "-m", mod, *args]
    print("Running:", " ".join(cmd))
    subprocess.check_call(cmd, cwd=str(PROJECT_ROOT))

print("OS:", platform.platform())
print("Python:", sys.version.split()[0])
print("DuckDB:", con.execute("SELECT version()").fetchone()[0])


PROJECT_ROOT: /Users/itnas/Documents/UQ/FisicaComputacional3/Estudiantes
OS: macOS-26.0.1-arm64-arm-64bit
Python: 3.9.6
DuckDB: v1.4.4


## 1) Localizar el archivo Raw

Nombre esperado (por el script oficial del curso): `pscomppars.csv`

**Regla del curso:** el Raw vive en `data/raw/` y NO se edita a mano.


In [2]:
EXPECTED = "pscomppars.csv"
raw_csv = RAW_DIR / EXPECTED

# Si no está con el nombre esperado, buscamos candidatos (para no bloquear la clase)
if not raw_csv.exists():
    candidates = list(RAW_DIR.glob("pscomppars*.csv")) + list(RAW_DIR.glob("*.csv"))
    candidates = [c for c in candidates if c.is_file()]
    if candidates:
        raw_csv = sorted(candidates)[0]
        print(f"No encontré {EXPECTED}. Encontré y usaré: {raw_csv.name}")
        print("   Recomendación: renombra a pscomppars.csv para estandarizar el curso.")
    else:
        raw_csv = None

raw_csv


PosixPath('/Users/itnas/Documents/UQ/FisicaComputacional3/Estudiantes/data/raw/pscomppars.csv')

## 2) (Opcional) Descargar el Raw con el script del curso

Si **tienes internet** y quieres descargar el Raw, ejecuta este comando en terminal **desde la raíz del proyecto**:

- Windows / mac / Linux:
`python -m src.ingest.download_exoplanets --format csv --limit 50000`

**Nota:** el notebook NO intenta descargar automáticamente (para evitar errores cuando no hay internet).


In [3]:
# Si no hay Raw, NO intentamos descargar automáticamente (para evitar fallos sin internet).
# Si quieres descargar desde clase (y tienes internet), pon DO_DOWNLOAD=True.
DO_DOWNLOAD = False

if raw_csv is None:
    if DO_DOWNLOAD:
        # Descarga oficial del curso (requiere internet). Escribe en data/raw/pscomppars.csv
        run_module("src.ingest.download_exoplanets", "--format", "csv", "--limit", "50000")
        raw_csv = RAW_DIR / "pscomppars.csv"
    else:
        raise FileNotFoundError(
            "No encontré ningún CSV en data/raw/.\n"
            "Solución A (recomendada): coloca pscomppars.csv en data/raw/.\n"
            "Solución B (con internet): cambia DO_DOWNLOAD=True y re-ejecuta esta celda."
        )

# Evidencia de trazabilidad
show(raw_csv), sha256_file(raw_csv)


({'exists': True,
  'path': 'C:\\Users\\hola-\\Documents\\FisicaComputacional3-Win\\de1-fundamentos-nasa-duckdb\\data\\raw\\pscomppars.csv',
  'size_bytes': 921168},
 '3c67b65ad2c3b65caddb80ddd1c400fc0d12a55f11cb623c15bdf9f7542c7b4d')

## 3) Crear VIEW Bronze/Raw

Creamos una vista para consultar el CSV sin modificarlo.


In [3]:
path = str(raw_csv).replace("\\", "/").replace("'", "''")  # Windows-safe
con.execute(f"""
CREATE OR REPLACE VIEW raw_ps AS
SELECT * FROM read_csv_auto('{path}')
""")

In [4]:
con.execute("SELECT COUNT(*) AS n_rows FROM raw_ps").fetchall()


[(6087,)]

## 4) Sanity checks mínimos (evidencia)

Haremos 3 checks:
1) filas y columnas
2) nulos en una columna clave
3) muestra de filas


In [5]:
# Check 1
n_rows = con.execute("SELECT COUNT(*) FROM raw_ps").fetchone()[0]
n_cols = con.execute("SELECT COUNT(*) FROM pragma_table_info('raw_ps')").fetchone()[0]
n_rows, n_cols

(6087, 16)

In [6]:
# 2) nulos en una columna clave típica
con.execute("SELECT COUNT(*) AS null_pl_name FROM raw_ps WHERE pl_name IS NULL").fetchall()

[(0,)]

In [7]:
# Check 3
con.execute("SELECT pl_name, hostname, discoverymethod, disc_year FROM raw_ps WHERE pl_name IS NOT NULL LIMIT 10").fetchall()


[('Kepler-1167 b', 'Kepler-1167', 'Transit', 2016),
 ('Kepler-1740 b', 'Kepler-1740', 'Transit', 2021),
 ('Kepler-1581 b', 'Kepler-1581', 'Transit', 2016),
 ('Kepler-644 b', 'Kepler-644', 'Transit', 2016),
 ('Kepler-1752 b', 'Kepler-1752', 'Transit', 2021),
 ('Kepler-280 c', 'Kepler-280', 'Transit', 2014),
 ('Kepler-1208 b', 'Kepler-1208', 'Transit', 2016),
 ('Kepler-263 c', 'Kepler-263', 'Transit', 2014),
 ('Kepler-1101 b', 'Kepler-1101', 'Transit', 2016),
 ('HD 168746 b', 'HD 168746', 'Radial Velocity', 2002)]

## 5) Guardar evidencia en artifacts (JSON)

Esto es evaluable y reproducible.


In [8]:
artifact = {
  "raw_file": str(raw_csv),
  "raw_sha256": sha256_file(raw_csv),
  "n_rows": n_rows,
  "n_cols": n_cols
}
out = ARTIFACTS_DIR / f"w01b_raw_evidence_{int(time.time())}.json"
out.write_text(json.dumps(artifact, indent=2), encoding="utf-8")
show(out)


{'exists': True,
 'path': '/Users/itnas/Documents/UQ/FisicaComputacional3/Estudiantes/artifacts/w01b_raw_evidence_1770177020.json',
 'size_bytes': 220}

## Entregable W01B
- `docs/w01b_checks.md`: pega 3 checks (query + resultado).
- `artifacts/w01b_raw_evidence_*.json`: hash + n_rows + n_cols.

## Tarea

- Agregar 1 sanity check extra y documentarlo en docs/w01b_checks.md.

    Ejemplos permitidos:

    - duplicados por pl_name

    - valores negativos en columnas numéricas

    - años fuera de rango (ej. < 1980 o > año actual)

- Registrar una decisión en docs/decisions_log.md (tu formato exacto) sobre trazabilidad:
Ejemplo real:


```markdown
- Fecha: 2025-12-21
- Decisión: Guardar SHA-256 del CSV raw en artifacts por cada ejecución.
- Razón: Detectar cambios invisibles del dato (DDIA reliability/operability).
- Alternativas: Confiar en el nombre del archivo (rechazada), usar solo fecha de descarga (rechazada).
- Evidencia (counts, EXPLAIN, timings): raw_sha256=..., n_rows=..., n_cols=...
```
